In [1]:
import openai
import json
import numpy as np
import os
import re
from dotenv import load_dotenv
import time
from tqdm import tqdm
load_dotenv()

MY_OPENAI_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=MY_OPENAI_KEY)
#최신버전 확인

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        book_data = json.load(f)
    return book_data


def get_embedding(text, model="text-embedding-3-large", retries=3):
    for attempt in range(retries):
        try:
            # 최신 OpenAI SDK v1 기준
            response = openai.embeddings.create(
                input=text,
                model=model
            )
            # 최신 버전의 응답 데이터 구조에 맞춤
            return response.data[0].embedding  
        
        except openai.APIConnectionError:
            print(f"⚠️ API 연결 오류 발생. {attempt+1}/{retries} 재시도 중...")
            time.sleep(2)  # 2초 대기 후 재시도
            
        except openai.RateLimitError:
            print(f"🚫 요청 제한 초과. {attempt+1}/{retries} 재시도 중...")
            time.sleep(5)  # 요청 제한 시에는 더 긴 시간 대기
            
        except openai.APIError as e:
            print(f"❌ OpenAI API 오류: {e}")
            break
        
        except Exception as e:
            print(f"❓ 예상치 못한 오류 발생: {e}")
            break
    
    return None


file_path = "../data/scraping/llm_output_openai.json" 
embedding_save_path = "../data/scraping/book_embeddings_openai.json" 


def embed_book_sentence(book_data):
    """
    책 데이터의 소개글을 임베딩합니다.
    """

    sentence = [book["sentence"] for book in book_data]
    hashtags = [
        [tag.replace("_", "") for tag in re.findall(r'#([\w가-힣_]+)', book["hashtags"])]
        for book in book_data
    ]
    letter = [book["letter"] for book in book_data]

    print(f"{len(sentence) = }")
    print(f"{len(hashtags) = }")
    print(f"len(letter): {len(letter)}")
    print(f"hashtags[0]: {hashtags[5]}")

    book_content = [
    f"{message} {letters} {' '.join(tags)}" for message, letters, tags in zip(sentence, letter, hashtags)
    ]

    print(len(book_content))
    print(book_content[5])

    embeddings = []
    for idx, text in enumerate(book_content):
        single_embedding = get_embedding(text)
        print(f"{idx} has completed")
        embeddings.append(single_embedding)

    return embeddings

# def save_embeddings(book_data, embeddings, save_path):
#     """
#     책 데이터와 임베딩 결과를 저장합니다.
#     """
#     ids = [book["isbn"] for book in book_data]
#     np.savez_compressed(save_path, ids=ids, embeddings=embeddings)

def save_embeddings_json(book_data, embeddings, save_path):
    """
    책 데이터와 임베딩을 JSON 형식으로 저장
    """
    data_to_save = {}

    for idx, (book, embedding) in enumerate(zip(book_data, embeddings)):
        if embedding is not None:  # None 값 필터링
            data_to_save[f"book{idx+1}"] = {
                "isbn": book["isbn"],  # 책의 ISBN (고유 식별자)
                "sentence": book["sentence"],  # 책 소개 문장
                "hashtags": book["hashtags"],
                "letter" : book["letter"],  # 해시태그
                "embedding": embedding  # 생성된 임베딩 벡터
            }

    # JSON 파일로 저장
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

    print(f"✅ JSON 파일 저장 완료: {save_path}")


In [2]:
book_data = load_json(file_path)
# book_data
book_data = list(book_data.values())

print(type(book_data))
print(book_data[0])


<class 'list'>
{'hashtags': '#조용한행복 #성장의아픔 #삶의지혜 #내면의평화 #인생수비력 #부정에서긍정으로 #실패의우아함 #도전의가치 #인생여백 #일상의소중함', 'letter': '당신이 아직 미처 알지 못하는 조용한 행복을 찾아가는 길을 제안합니다. 그 길은 울 일이 없는 하루를 만들어가는 과정에서 발견될 수 있습니다. 실패와 절망, 그리고 눈물이 있을지라도 그것이 진정한 인생이며, 그 중에서도 행복을 찾아내는 과정을 함께 나누고 싶습니다. 나아가 행복은 누군가에게 주어지는 것이 아니라, 스스로 찾아가는 것이라는 것을 기억해주시길 바랍니다.', 'isbn': 9791169851053, 'sentence': '조용한 행복이란, 울 일이 없는 하루, 나쁜 일이 없는 일상입니다. 여백이 있는 인생이야말로 진정한 행복입니다.'}


In [3]:
# 책 소개글을 임베딩합니다.
print("Embedding book content...")
embeddings = embed_book_sentence(book_data)


# 임베딩 결과를 저장합니다.
print(f"Saving embeddings to {embedding_save_path}...")
save_embeddings_json(book_data, embeddings, embedding_save_path)

print("Embedding and saving complete.")

Embedding book content...
len(sentence) = 1781
len(hashtags) = 1781
len(letter): 1781
hashtags[0]: ['김창완', '찌그러져도동그라미입니다', '삶의진리', '일상의소중함', '감사의마음', '현재진행형아티스트', '잔잔한위로', '인생의온도', '희망의소소함', '모든일에는내일이있다']
1781
"한 움큼의 인생, 그 안에 찌그러진 동그라미가 있습니다." 찌그러져도 동그라미인 우리의 삶, 그리고 그 속의 작은 희망들을 찾아냅시다. 모든 새벽이 밝아오듯, 우리의 일상도 결국은 동그라미가 될 것입니다. 조금은 찌그러져도 그것이 바로 우리의 삶, 그리고 우리의 희망입니다. 함께 그림을 그려갑시다, 동그라미를. 언제나 여러분과 함께하는 시간을 기대하며. 김창완 찌그러져도동그라미입니다 삶의진리 일상의소중함 감사의마음 현재진행형아티스트 잔잔한위로 인생의온도 희망의소소함 모든일에는내일이있다
0 has completed
1 has completed
2 has completed
3 has completed
4 has completed
5 has completed
6 has completed
7 has completed
8 has completed
9 has completed
10 has completed
11 has completed
12 has completed
13 has completed
14 has completed
15 has completed
16 has completed
17 has completed
18 has completed
19 has completed
20 has completed
21 has completed
22 has completed
23 has completed
24 has completed
25 has completed
26 has completed
27 has completed
28 has completed
29 has completed
30 has complete